In [19]:
import zipfile

with zipfile.ZipFile("/L01.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/script")  # Giải nén vào thư mục này

In [24]:
import os
import json
import pandas as pd

script_folder = "/content/script/L01"  # <- cập nhật đúng path
all_segments = []

for filename in os.listdir(script_folder):
    if filename.endswith("_script.json"):
        video_id = filename.replace("_script.json", "")
        file_path = os.path.join(script_folder, filename)

        with open(file_path, "r", encoding="utf-8") as f:
            try:
                data = json.load(f)
                if isinstance(data, list):
                    for seg in data:
                        if all(k in seg for k in ["start", "end", "text"]):
                            all_segments.append({
                                "video": video_id,
                                "start": seg["start"],
                                "end": seg["end"],
                                "text": seg["text"]
                            })
            except Exception as e:
                print(f"❌ Lỗi đọc {filename}: {e}")

# Tạo DataFrame
df = pd.DataFrame(all_segments)
print(f"✅ Đã load {len(df)} đoạn từ {df['video'].nunique()} video.")
df.head()

✅ Đã load 941 đoạn từ 8 video.


,video,start,end,text
0,L01_V008,2.400000,18.400000,Kính chào quý vị! Chúng tôi rất vui được gặp l...
1,L01_V008,22.400000,24.566667,Lô gạo phát thải thấp đầu tiên của Việt Nam xu...
2,L01_V008,31.200000,33.066667,Tuyên án 31 bị cáo lừa đảo bằng công nghệ cao ...
3,L01_V008,37.766667,58.133333,Trong số 100 ca nhân xuất sắc được vinh danh l...
4,L01_V008,58.166667,61.433333,"Trung Bình, mỗi người này đã hiến máu 48 lần, ..."


In [25]:
!pip install -q faiss-cpu sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.7 MB/s eta 0:00:00


In [26]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

model = SentenceTransformer("all-MiniLM-L6-v2")

# Encode tất cả đoạn text
embeddings = model.encode(df["text"].tolist(), show_progress_bar=True)

# Tạo FAISS index có thể truy ID ngược lại
index = faiss.IndexIDMap(faiss.IndexFlatL2(embeddings.shape[1]))
index.add_with_ids(np.array(embeddings), np.arange(len(df)))

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/30 [00:00<?, ?it/s]

In [28]:
def search_transcript(query, top_k=5):
    query_vec = model.encode([query])
    D, I = index.search(np.array(query_vec), top_k)

    print(f"\n🔍 Kết quả cho: “{query}”\n")
    for idx in I[0]:
        row = df.iloc[idx]
        print(f"- 📺 {row['video']} | 🕒 {row['start']:.2f}s → {row['end']:.2f}s")
        print(f"  📜 {row['text']}\n")

# Ví dụ tìm kiếm
search_transcript("thể thao")


🔍 Kết quả cho: “thể thao”

- 📺 L01_V003 | 🕒 1281.97s → 1287.57s
  📜 Cuộc thi sức được khuyến khích tại địa phương không chỉ bởi sự tôn vinh, tính độc đáo, tinh thật thể thao, mà còn đem lại sự nhộn nghiệp cho cua lạc nhỏ, vắng người phùng Tây Bắc.

- 📺 L01_V003 | 🕒 1247.43s → 1248.83s
  📜 găng tay, đua thể thao, áo len và bố bảo hiểm. Các thế đua rất hào cứng tham gia cuộc đua sức bền đường bãi xanh vô địch nhất thế giới dành cho máy cắt cỏ kéo xài trong suốt 24 chiều.

- 📺 L01_V003 | 🕒 1074.67s → 1077.57s
  📜 Mẹo chỉ có thế thôi.. .

- 📺 L01_V004 | 🕒 144.60s → 148.53s
  📜 Ở trong này tát ra thì ở trong này nó ngập. Bây giờ ở ngoài sông thì nước nó nhiều quá. Nó ngập vậy đó mà nếu mà nó để tới nước tự nhiên nó giật thì nó thúi ghế, nó thúi góc nó chết.

- 📺 L01_V008 | 🕒 739.27s → 741.23s
  📜 Ở Tân phố, nơi hoa hồng là hoàng hậu.

